In [ ]:
# Load dependencies

import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from scipy.stats.stats import pearsonr
from matplotlib import pyplot as plt

In [ ]:
# Read in the structured correlation data
# Update filename based on epoch duration

corr_df = pd.read_csv("preprocessed_interpolate_bandpass_.1_20/merged_corr_df_1s_epoch.csv")
len(corr_df)

In [ ]:
# This filters out extreme values that are > +/-.3
# This is a brute-force way of excluding outliers
filter_index = (corr_df[["TP10", "AF7", "AF8", "TP9"]] > 0.4).any(1)  | (corr_df[["TP10", "AF7", "AF8", "TP9"]] < -0.4).any(1) | (corr_df["Signal Drop"] == 1)

# Print how many are filtered out
filter_index.sum()

In [ ]:
# If you want to remove outliers identified above, do it by uncommenting this code
corr_df = corr_df[~filter_index]
len(corr_df)

In [ ]:
# Count the unique number of subjects in the 'subj' column of the 'corr_df' dataframe
# This is achieved by selecting the 'subj' column, calling .unique() to get an array of unique values,
# and then accessing the first element of the .shape attribute, which represents the number of unique subjects

corr_df.subj.unique().shape[0]

In [ ]:
# Get some insights into the data
sns.pairplot(corr_df[["TP9", "AF7", "AF8", "TP10", "Performance", "Time (sec)"]])
plt.show()

In [ ]:
# Plot boxplot of electrode ISC by session
fig, axs = plt.subplots(2, 2, figsize=(8.5, 8.5), dpi=300)  # Set figure size to 8.5x8.5 inches

channels = ["AF7", "AF8", "TP9", "TP10"]
# Define custom color palette, from color brewer
color_palette = {1: '#4d004b', 2: '#88419d', 3: '#8c96c6'}

for i, channel in enumerate(channels):
    sns.boxplot(x="ses", y=channel, data=corr_df, ax=axs[i//2, i%2],
                palette=color_palette)  # Use the custom palette for box plots
    axs[i//2, i%2].set_ylabel(channel)  # Explicitly set y-axis label to the channel name
    axs[i//2, i%2].set_ylim(-0.4, 0.4)  # Set the y-axis range to be from -0.3 to 0.3

# Ensure the background of the figure is white
fig.patch.set_facecolor('white')

plt.tight_layout(pad=2.0)  # Increase padding between plots for clarity
plt.savefig("plots/isc_by_session_boxplot.png", bbox_inches='tight', facecolor=fig.get_facecolor())  # Save the figure as a PNG file
plt.show()


In [ ]:
# Test if ISCs in a given electrode increase as a function of session
# This would test if participant ISCs increase over time

# Fit a mixed model
# This formula specifies a model with:
    # ISC for a given electrode as the DV
    # ses as a fixed effect
    # subject as a random intercepts

# This model is fir for ISCs in the AF7 electrode
model = smf.mixedlm("AF7 ~ ses", corr_df, groups=corr_df["subj"])
result = model.fit()

print(result.summary())

In [ ]:
# Test if ISCs in a given electrode increase as a function of session
# This would test if participant ISCs increase over time

# Fit a mixed model
# This formula specifies a model with:
    # ISC for a given electrode as the DV
    # ses as a fixed effect
    # subject as a random intercepts

# This model is fir for ISCs in the AF8 electrode
model = smf.mixedlm("AF8 ~ ses", corr_df, groups=corr_df["subj"])
result = model.fit()

print(result.summary())

In [ ]:
# Test if ISCs in a given electrode increase as a function of session
# This would test if participant ISCs increase over time

# Fit a mixed model
# This formula specifies a model with:
    # ISC for a given electrode as the DV
    # ses as a fixed effect
    # subject as a random intercepts

# This model is fir for ISCs in the TP9 electrode
model = smf.mixedlm("TP9 ~ ses", corr_df, groups=corr_df["subj"])
result = model.fit()

print(result.summary())

In [ ]:
# Test if ISCs in a given electrode increase as a function of session
# This would test if participant ISCs increase over time

# Fit a mixed model
# This formula specifies a model with:
    # ISC for a given electrode as the DV
    # ses as a fixed effect
    # subject as a random intercepts

# This model is fir for ISCs in the TP10 electrode
model = smf.mixedlm("TP10 ~ ses", corr_df, groups=corr_df["subj"])
result = model.fit()

print(result.summary())

In [ ]:
# Fit a complex mixed model that deals with sessions nested within sujects
# This model:
    # "Performance ~ Electrode" defines "Performance" as the DV and"AF7" as an independent variable (fixed effect)
    # groups=corr_df["subj"] specifies that the random intercepts should vary by "subj" to account for the non-independence of observations within subjects
    #re_formula="1" indicates that there is a random intercept for each subject
    # vc_formula={"ses": "0 + C(ses)"} specifies a variance component for "ses", treating sessions as a categorical variable (with C(ses)) and allowing for varying intercepts by "ses" without a slope
# This may not converge, so we fit simpler models below

# Define the models
model_af7 = smf.mixedlm("Performance ~ AF7", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_af8 = smf.mixedlm("Performance ~ AF8", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp9 = smf.mixedlm("Performance ~ TP9", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp10 = smf.mixedlm("Performance ~ TP10", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

# Fit the model
results_af7 = model_af7.fit()
results_af8 = model_af8.fit()
results_tp9 = model_tp9.fit()
results_tp10 = model_tp10.fit()

# Print the summary of the model fit
print(results_af7.summary())
print(results_af8.summary())
print(results_tp9.summary())
print(results_tp10.summary())

In [ ]:
# Plot the mixed model of electrode on performance

# Prepare the grid for plotting
fig, axs = plt.subplots(2, 2, figsize=(8.5, 8.5), dpi=300)  # Adjust figsize to 8.5 x 8.5 inches

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']

for i, electrode in enumerate(electrodes):
    # Fit the model for the current electrode
    model = smf.mixedlm(f"Performance ~ {electrode}", data=corr_df, groups=corr_df["subj"], re_formula="1",
                        vc_formula={"ses": "0 + C(ses)"}).fit()
    
    # Get predicted values for a range of electrode values
    electrode_values = np.linspace(corr_df[electrode].min(), corr_df[electrode].max(), 100)
    predicted_performance = model.params['Intercept'] + model.params[electrode] * electrode_values
    
    # Determine the subplot to use
    ax = axs[i // 2, i % 2]
    
    # Plot the observed data using the specified color
    sns.scatterplot(x=electrode, y='Performance', data=corr_df, color='#8c96c6', alpha=0.6, ax=ax)
    
    # Plot the fitted line using the specified color
    ax.plot(electrode_values, predicted_performance, color='#4d004b', label='Fitted Line')
    
    # Set titles and labels
    ax.set_xlabel(electrode)
    ax.set_ylabel('Performance')
    ax.set_title(f'Mixed Model Fit: Performance ~ {electrode}')
    ax.legend()

# Ensure the background of the figure is white
fig.patch.set_facecolor('white')

plt.tight_layout()
plt.savefig('plots/model_fits_performance.png', format='png', dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')  # Save the figure
plt.show()


In [ ]:
# Rename the time variable for easier modeling

corr_df = corr_df.rename(columns={'Time (sec)': 'Time'})

In [ ]:
# Now fit the same model for time

# Fit a complex mixed model that deals with sessions nested within sujects
# This model:
    # "Performance ~ Electrode" defines "Performance" as the DV and"AF7" as an independent variable (fixed effect)
    # groups=corr_df["subj"] specifies that the random intercepts should vary by "subj" to account for the non-independence of observations within subjects
    #re_formula="1" indicates that there is a random intercept for each subject
    # vc_formula={"ses": "0 + C(ses)"} specifies a variance component for "ses", treating sessions as a categorical variable (with C(ses)) and allowing for varying intercepts by "ses" without a slope
# This may not converge, so we fit simpler models below

# Define the models
model_af7 = smf.mixedlm("Time ~ AF7", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_af8 = smf.mixedlm("Time ~ AF8", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp9 = smf.mixedlm("Time ~ TP9", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp10 = smf.mixedlm("Time ~ TP10", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

# Fit the model
results_af7 = model_af7.fit()
results_af8 = model_af8.fit()
results_tp9 = model_tp9.fit()
results_tp10 = model_tp10.fit()

# Print the summary of the model fit
print(results_af7.summary())
print(results_af8.summary())
print(results_tp9.summary())
print(results_tp10.summary())

In [ ]:
# Plot the mixed model of electrode on performance

# Prepare the grid for plotting
fig, axs = plt.subplots(2, 2, figsize=(8.5, 8.5), dpi=300)  # Adjust figsize to 8.5 x 8.5 inches

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']

for i, electrode in enumerate(electrodes):
    # Fit the model for the current electrode
    model = smf.mixedlm(f"Time ~ {electrode}", data=corr_df, groups=corr_df["subj"], re_formula="1",
                        vc_formula={"ses": "0 + C(ses)"}).fit()
    
    # Get predicted values for a range of electrode values
    electrode_values = np.linspace(corr_df[electrode].min(), corr_df[electrode].max(), 100)
    predicted_performance = model.params['Intercept'] + model.params[electrode] * electrode_values
    
    # Determine the subplot to use
    ax = axs[i // 2, i % 2]
    
    # Plot the observed data using the specified color
    sns.scatterplot(x=electrode, y='Time', data=corr_df, color='#8c96c6', alpha=0.6, ax=ax)
    
    # Plot the fitted line using the specified color
    ax.plot(electrode_values, predicted_performance, color='#4d004b', label='Fitted Line')
    
    # Set titles and labels
    ax.set_xlabel(electrode)
    ax.set_ylabel('Time')
    ax.set_title(f'Mixed Model Fit: Time ~ {electrode}')
    ax.legend()

# Ensure the background of the figure is white
fig.patch.set_facecolor('white')

plt.tight_layout()
plt.savefig('plots/model_fits_time.png', format='png', dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')  # Save the figure
plt.show()


In [ ]:
corr_df

In [ ]:
# Now fit the same model for cognitive trust

# Fit a complex mixed model that deals with sessions nested within sujects
# This model:
    # "Cognitive Trust ~ Electrode" defines "Cognitive Trust" as the DV and"AF7" as an independent variable (fixed effect)
    # groups=corr_df["subj"] specifies that the random intercepts should vary by "subj" to account for the non-independence of observations within subjects
    #re_formula="1" indicates that there is a random intercept for each subject
    # vc_formula={"ses": "0 + C(ses)"} specifies a variance component for "ses", treating sessions as a categorical variable (with C(ses)) and allowing for varying intercepts by "ses" without a slope
# This may not converge, so we fit simpler models below

# Define the models
model_af7 = smf.mixedlm("cognitive_trust ~ AF7", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_af8 = smf.mixedlm("cognitive_trust ~ AF8", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp9 = smf.mixedlm("cognitive_trust ~ TP9", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp10 = smf.mixedlm("cognitive_trust ~ TP10", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

# Fit the model
results_af7 = model_af7.fit()
results_af8 = model_af8.fit()
results_tp9 = model_tp9.fit()
results_tp10 = model_tp10.fit()

# Print the summary of the model fit
print(results_af7.summary())
print(results_af8.summary())
print(results_tp9.summary())
print(results_tp10.summary())

In [ ]:
# Plot the mixed model of electrode on cognitive_trust

# Prepare the grid for plotting
fig, axs = plt.subplots(2, 2, figsize=(8.5, 8.5), dpi=300)  # Adjust figsize to 8.5 x 8.5 inches

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']

for i, electrode in enumerate(electrodes):
    # Fit the model for the current electrode
    model = smf.mixedlm(f"cognitive_trust ~ {electrode}", data=corr_df, groups=corr_df["subj"], re_formula="1",
                        vc_formula={"ses": "0 + C(ses)"}).fit()
    
    # Get predicted values for a range of electrode values
    electrode_values = np.linspace(corr_df[electrode].min(), corr_df[electrode].max(), 100)
    predicted_performance = model.params['Intercept'] + model.params[electrode] * electrode_values
    
    # Determine the subplot to use
    ax = axs[i // 2, i % 2]
    
    # Plot the observed data using the specified color
    sns.scatterplot(x=electrode, y='cognitive_trust', data=corr_df, color='#8c96c6', alpha=0.6, ax=ax)
    
    # Plot the fitted line using the specified color
    ax.plot(electrode_values, predicted_performance, color='#4d004b', label='Fitted Line')
    
    # Set titles and labels
    ax.set_xlabel(electrode)
    ax.set_ylabel('Time')
    ax.set_title(f'Mixed Model Fit: Cognitive Trust ~ {electrode}')
    ax.legend()

# Ensure the background of the figure is white
fig.patch.set_facecolor('white')

plt.tight_layout()
plt.savefig('plots/model_fits_cognitive_trust.png', format='png', dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')  # Save the figure
plt.show()


In [ ]:
# Now fit the same model for affective trust

# Fit a complex mixed model that deals with sessions nested within sujects
# This model:
    # "Cognitive Trust ~ Electrode" defines "Cognitive Trust" as the DV and"AF7" as an independent variable (fixed effect)
    # groups=corr_df["subj"] specifies that the random intercepts should vary by "subj" to account for the non-independence of observations within subjects
    #re_formula="1" indicates that there is a random intercept for each subject
    # vc_formula={"ses": "0 + C(ses)"} specifies a variance component for "ses", treating sessions as a categorical variable (with C(ses)) and allowing for varying intercepts by "ses" without a slope
# This may not converge, so we fit simpler models below

# Define the models
model_af7 = smf.mixedlm("affective_trust ~ AF7", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_af8 = smf.mixedlm("affective_trust ~ AF8", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp9 = smf.mixedlm("affective_trust ~ TP9", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp10 = smf.mixedlm("affective_trust ~ TP10", data=corr_df, groups=corr_df["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

# Fit the model
results_af7 = model_af7.fit()
results_af8 = model_af8.fit()
results_tp9 = model_tp9.fit()
results_tp10 = model_tp10.fit()

# Print the summary of the model fit
print(results_af7.summary())
print(results_af8.summary())
print(results_tp9.summary())
print(results_tp10.summary())

In [ ]:
# Plot the mixed model of electrode on affective_trust

# Prepare the grid for plotting
fig, axs = plt.subplots(2, 2, figsize=(8.5, 8.5), dpi=300)  # Adjust figsize to 8.5 x 8.5 inches

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']

for i, electrode in enumerate(electrodes):
    # Fit the model for the current electrode
    model = smf.mixedlm(f"affective_trust ~ {electrode}", data=corr_df, groups=corr_df["subj"], re_formula="1",
                        vc_formula={"ses": "0 + C(ses)"}).fit()
    
    # Get predicted values for a range of electrode values
    electrode_values = np.linspace(corr_df[electrode].min(), corr_df[electrode].max(), 100)
    predicted_performance = model.params['Intercept'] + model.params[electrode] * electrode_values
    
    # Determine the subplot to use
    ax = axs[i // 2, i % 2]
    
    # Plot the observed data using the specified color
    sns.scatterplot(x=electrode, y='affective_trust', data=corr_df, color='#8c96c6', alpha=0.6, ax=ax)
    
    # Plot the fitted line using the specified color
    ax.plot(electrode_values, predicted_performance, color='#4d004b', label='Fitted Line')
    
    # Set titles and labels
    ax.set_xlabel(electrode)
    ax.set_ylabel('Time')
    ax.set_title(f'Mixed Model Fit: Affective Trust ~ {electrode}')
    ax.legend()

# Ensure the background of the figure is white
fig.patch.set_facecolor('white')

plt.tight_layout()
plt.savefig('plots/model_fits_affective_trust.png', format='png', dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')  # Save the figure
plt.show()